In [1]:
#Import libraries
import pandas as pd
import numpy as np

#Read in the data
df=pd.read_csv("https://raw.githubusercontent.com/statzenthusiast921/house-hunters-international/main/data/hhi.csv",encoding='latin-1')
print(df.shape)

(2419, 14)


In [2]:
#Format origin and destinations a little more clearly
df['Origin'] = np.where(
    df['MoveFromCity'].str.contains(',') == True, df['MoveFromCity'], df['MoveFromCity'] + ', ' + df['MoveFromCountry']
)
df['Destination'] = np.where(
    df['MoveToCity'].str.contains(',') == True, df['MoveToCity'], df['MoveToCity'] + ', ' + df['MoveToCountry']
)

In [3]:
#Clean up a few columns
del df['NumWords'], df['Entities']

df = df[df['NumBlanks']<4]
df['NumBlanks'].value_counts()

0    1105
2     841
1     354
3      94
Name: NumBlanks, dtype: int64

In [4]:
#Create geo-categories to separate analysis types
conditions  = [
               df['NumBlanks'] == 0, 
               df['NumBlanks'] == 3,
               (df['NumBlanks']== 2) & (df['MoveFromCity'].isna() == True) & (df['MoveToCity'].isna() == True),
               (df['NumBlanks']== 2) & (df['MoveFromCity'].isna() == True) & (df['MoveFromCountry'].isna() == True),
               (df['NumBlanks']== 2) & (df['MoveToCity'].isna() == True) & (df['MoveToCountry'].isna() == True),
               (df['NumBlanks']== 1) & (df['MoveFromCity'].isna() == True),
               (df['NumBlanks']== 1) & (df['MoveFromCountry'].isna() == True),
               (df['NumBlanks']== 1) & (df['MoveToCity'].isna() == True),
               (df['NumBlanks']== 1) & (df['MoveToCountry'].isna() == True),
              ]
choices     = ["All",
               'Destination Country',
               'Country to Country',
               'All Destination',
               'All Origin',
               'Missing Origin City',
               'Missing Origin Country',
               'Missing Destination City',
               'Missing Destination Country'
              ]
    
df["GeoCategory"] = np.select(conditions, choices, default=np.nan)
df['GeoCategory'].value_counts()

All                         1105
All Destination              801
Missing Origin City          259
Missing Destination City      95
Destination Country           94
Country to Country            38
All Origin                     2
Name: GeoCategory, dtype: int64

In [5]:
df = df.reset_index()
df.head(2)

,index,ep_summary,air_date,ep_nums,ep_title,episode,season,year,MoveFromCity,MoveFromCountry,MoveToCity,MoveToCountry,NumBlanks,Origin,Destination,GeoCategory
0,0,Perry and Cheryl Meriot are Edmonton Alberta n...,21-Feb-10,S12E11,Canadian Couple Buys a Vacation Home in Punta ...,11.0,12.0,2010.0,Edmonton,Canada,Punta Cana,Dominican Republic,0,"Edmonton, Canada","Punta Cana, Dominican Republic",All
1,1,Ken and Donna Richardson live in Conway South ...,27-Jan-10,S12E12,South Carolina Retirees Buy 2nd Home in Costa ...,12.0,12.0,2010.0,"Conway, South Carolina",United States,Nosara,Costa Rica,0,"Conway, South Carolina","Nosara, Costa Rica",All


### Pulling out latitude and longitude from city names

In [7]:
from geopy import distance
from geopy.geocoders import Photon
from geopy.geocoders import Nominatim

df1 = df[df['GeoCategory']== "All"]

In [8]:
#Example using Photon
geolocator = Photon(user_agent="measurements")
address='London, England'
location = geolocator.geocode(address)
print(location.address)
print((location.latitude, location.longitude))

London, England, United Kingdom
(51.5073359, -0.12765)


In [17]:
#test1 = df1[0:10]
#geolocator = Nominatim(user_agent="measurements", timeout=3)
geolocator = Photon(user_agent="measurements",timeout=None)


def extract_coordinates(location):
    geocode_result = geolocator.geocode(location, language='en')
    if geocode_result is not None:
        latitude = geocode_result.latitude
        longitude = geocode_result.longitude
        return latitude, longitude
    else:
        return None, None


df1[['latitude', 'longitude']] = df1['Origin'].apply(extract_coordinates).apply(pd.Series)
df1.head(3)

/var/folders/yr/521qh_vn01n7j762jny9bmhw0000gn/T/ipykernel_99783/2413742714.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[['latitude', 'longitude']] = df1['Origin'].apply(extract_coordinates).apply(pd.Series)
/var/folders/yr/521qh_vn01n7j762jny9bmhw0000gn/T/ipykernel_99783/2413742714.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[['latitude', 'longitude']] = df1['Origin'].apply(extract_coordinates).apply(pd.Series)


,index,ep_summary,air_date,ep_nums,ep_title,episode,season,year,MoveFromCity,MoveFromCountry,MoveToCity,MoveToCountry,NumBlanks,Origin,Destination,GeoCategory,loc_info_orig,loc_info_dest,latitude,longitude
0,0,Perry and Cheryl Meriot are Edmonton Alberta n...,21-Feb-10,S12E11,Canadian Couple Buys a Vacation Home in Punta ...,11.0,12.0,2010.0,Edmonton,Canada,Punta Cana,Dominican Republic,0,"Edmonton, Canada","Punta Cana, Dominican Republic",All,"(Edmonton, Alberta, Canada, (53.5462055, -113....","(Punta Cana, 23300, Salvaleón de Higüey, La Al...",53.546205,-113.491241
1,1,Ken and Donna Richardson live in Conway South ...,27-Jan-10,S12E12,South Carolina Retirees Buy 2nd Home in Costa ...,12.0,12.0,2010.0,"Conway, South Carolina",United States,Nosara,Costa Rica,0,"Conway, South Carolina","Nosara, Costa Rica",All,"(Conway, 29562, South Carolina, United States,...","(Nosara, 50206, Provincia Guanacaste, Costa Ri...",33.836003,-79.047814
2,2,Dan and Andrea Layden (30's) are history buffs...,3-Feb-10,S12E08,Dan and Andrea Want a Family-Friendly Vacation...,8.0,12.0,2010.0,Philadelphia,United States,Abruzzo,Italy,0,"Philadelphia, United States","Abruzzo, Italy",All,"(Philadelphia, Pennsylvania, United States, (3...","(Abruzzo, Italy, (42.227681, 13.854983))",39.952724,-75.163526


In [18]:
#df1.to_csv('data_w_lat_lon.csv', encoding='utf-8', index=False)

### Check if coordinates make sense by plotting on map

In [23]:
import folium

map_osm = folium.Map()

df1.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]], 
                                              radius=10)
                                             .add_to(map_osm), axis=1)

map_osm